# Import libraries

In [1]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':'tight'}
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from numpy.random import random, seed
import pandas as pd
from time import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# 1. (40 pts) Define functions
### Define the following functions

In [6]:
def one_hot_encode(labels):
    """
        Turn single digit numerical value classes into length 10 vectors...
        1 for the positive class and 0's for the other 9 negative classes.
    """
    
    min_l, max_l = min(labels), max(labels)
    one_hot_map = {k: [0 if i < k else 1 if i == k else 0 \
                              for i in range(min_l, max_l+1)] \
                      for k in range(min_l, max_l+1)}
    
    return np.matrix([one_hot_map[label[0]] for label in labels]).reshape(10, -1)

def sigmoid(z):
    """
        Squishes the output of our hypothesis function into the range (0, 1).
    """
    
    return 1./(1. + np.exp(-z))

def softmax(outputs):
    """
        Produce confidence probabilities for the output classes using the output of the
        second-from-last layer.
    """
    
    exp = np.exp(outputs)
    return exp/exp.sum(axis=0)

    
def log_loss(output, labels):
    """
        Computes the multi-class log loss when provided with the output from the softmax
        group and the corresponding labeled data
    """
    
    return (1/len(labels.T)) * -np.sum(np.dot(labels.T, np.log(output)))
    
def forward(net, X):
    """
        Perform forward propagation through our neural network. Apply sigmoid to
        the first weighted sum; apply softmax to the second weighted sum.
    """
    
    # Retrieve weights from the network and reinitialize the cached activations
    W1 = net['weights'][0]
    W2 = net['weights'][1]
    net['activations'] = []
           
    # Perform forward pass while appyling bias terms to the input and a1 layers
    X_bias = np.concatenate((X, np.ones((X.shape[0], 1))), axis=1)   
    a1 = sigmoid(np.dot(W1, X_bias.T))                               
    a1_bias = np.concatenate((a1, np.ones((1, X.shape[0]))), axis=0) 
    a2 = softmax(np.dot(W2, a1_bias))                                
    
    # Cache input data and activations
    net['input'] = X  
    net['activations'].append(a1)
    net['activations'].append(a2)
    
    return a2
    
def backward(net, labels):
    """
        Perform back propagation through our neural network. Compute error derivatives
        w.r.t. weights.
    """
    
    # Retrieve input data, cached activations, and weights from the network
    X = net['input']
    a1 = net['activations'][0]
    a2 = net['activations'][1]
    W1 = net['weights'][0][:, :-1]
    b1 = net['weights'][0][:, -1].reshape(-1, 1)
    W2 = net['weights'][1][:, :-1]
    b2 = net['weights'][1][:, -1].reshape(-1, 1)

    # Compute loss function derivatives w.r.t. parameters and biases
    dL_dW2 = (1/len(X)) * np.dot((a2-labels), a1.T)
    dL_db2 = (1/len(X)) * np.sum((a2-labels), axis=1, keepdims=True)
    dL_dW1 = (1/len(X)) * np.dot(np.dot(np.dot(W2.T, (a2-labels)), np.dot((1-a1).T, a1)), X)
    dL_db1 = (1/len(X)) * np.sum(np.dot(np.dot(W2.T, (a2-labels)), np.dot((1-a1).T, a1)), axis=1, keepdims=True)
    
    return dL_dW1, dL_db1, dL_dW2, dL_db2
        
def gradient_descent(net, X, labels, alpha):
    """
        Use loss function derivatives w.r.t weights to continually improve the
        performace of our network.
    """
      
    def update_weights(weights, gradients):
        W1, b1 = weights[0][:, :-1], weights[0][:, -1].reshape(-1, 1)
        W2, b2 = weights[1][:, :-1], weights[1][:, -1].reshape(-1, 1)

        W1 -= alpha * gradients[0]
        b1 -= alpha * gradients[1]
        W2 -= alpha * gradients[2]
        b2 -= alpha * gradients[3]  

    new_cost, old_cost = log_loss(forward(net, X), labels), float('inf')    
    beginning = start = time()
    count = 0
    
    print('Initial cost:', new_cost)
    
    while new_cost < old_cost and abs(old_cost - new_cost) > 1e-5:
        update_weights(net['weights'], backward(net, labels))
        old_cost = new_cost
        new_cost = log_loss(forward(net, X), labels)
        if time() - start > 20:
            print('\t', new_cost)
            start = time()
        count += 1
    
    end = time() - beginning
    train_eval = log_loss(forward(net, X), labels)
        
    print('Final model has a log loss of {} -- achieved in {} iterations in {} seconds'.format(
            train_eval, count, end))
    
    return ()
    
def save_weights(weights):
    """
        Store learned model parameters
    """
    
    with open('nn_weights.npy', 'wb') as f:
        np.save(f, weights)
        
def load_weights():
    """
        If a weights file is present, load pretrained model parameters
    """
    
    try:
        with open('nn_weights.npy', 'rb') as f:
            return np.load(f)
    except:
        raise 'No weights file found'

# 2. (5 pts) Split data
### Split training and testing data into x and y sets. Input has columns from 0 to 399 and ouput has a column 'y'

In [7]:
train = pd.read_csv('ex3_train.csv')
test = pd.read_csv('ex3_test.csv')

x_train = train.iloc[:, :-1].as_matrix()
x_test = test.iloc[:, :-1].as_matrix()

# Min-max scale inputs to facilitate convergence
x_train = (x_train - x_train.min())/(x_train.max() - x_train.min())
x_test = (x_test - x_test.min())/(x_test.max() - x_test.min())

# One hot encode labels
y_train = one_hot_encode(train.iloc[:, -1].values.reshape(-1, 1))
y_test = one_hot_encode(test.iloc[:, -1].values.reshape(-1, 1))

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:8: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


# 3. (5 pts) Initialize parameters
### Use np.random.seed(1) when initializing weight coefficients. Set bias terms to 0.

In [8]:
# Form weight matrices by concatenating zeroed bias weights to randomly initialized parameter weights
seed(1); W1 = np.concatenate((random((25, x_train.shape[1])), np.zeros((25, 1))), axis=1)
seed(1); W2 = np.concatenate((random((y_train.shape[0], 25)), np.zeros((y_train.shape[0], 1))), axis=1)

# 4. (20 pts) Neural network model with one hidden layer
### Build a neural network model, using the training set, with an input layer of 400 neurons, one hidden layer of 25 neurons, and an output layer of 10 neurons.

In [9]:
"""
    [X: (n,400)+bias -> W1: (25,401) -> sig: (25,n)+bias -> W2: (10,26) -> softmax: (10,n)] -> Loss: -sum(t*logp)
"""

net = {'weights': [W1, W2],
       'activations': []
      }

try:
    net['weights'] = load_weights()
    print('Loaded precomputed weights')
except:
    print('Could not find precomputed weights...training model parameters from scratch...')
    gradient_descent(net, x_train, y_train, alpha=0.5)
    save_weights(net['weights'])

Could not find precomputed weights...training model parameters from scratch...
Initial cost: 10637.5151742


ValueError: operands could not be broadcast together with shapes (25,3500) (3500,3500) 

# 5. (10 pts) Predictions
### Predict digits using the softmax function. Calculate the accuracy of the predictions using both the training and testing data.

In [ ]:
train_eval = log_loss(forward(net, x_train), y_train)
test_eval = log_loss(forward(net, x_test), y_test)
print('Training: {}\nTesting: {}'.format(training_eval, testing_eval))

# 6. (20 pts) Optimization
### Optimize the model using a variety of learning rates and number of iterations. Plot the cost over the number of iterations with different learning rates for the training set. Print the optimized accuracy for the test set.

In [ ]:
for a in [1, 0.5, 0.1]:
    gradient_descent(net, x_train, y_train, alpha=a)
    train_eval = log_loss(forward(net, x_train), y_train)
    test_eval = log_loss(forward(net, x_test), y_test)
    scores.append((train_eval, test_eval, a))